# Client


In [1]:
from eventbus.event_net import Bridge

import asyncio
import json
import aiohttp
from eventbus.event_net.transport import AioTransport
from eventbus import bus



ws_url = "http://localhost:5000/ws"

CLIENT_ID = "client1"

cookies = { "leaf-token": CLIENT_ID }

bus.LEAF_ID = CLIENT_ID

async def sender_task():
    for i in range(3):
        event = { "topic": "?echo", "src": bus.LEAF_ID, "dst": "server", "data": i }
        await bus.emit(event) 
        await asyncio.sleep(4)
    await bus.emit({ "topic": "!bye", "src": bus.LEAF_ID, "dst": "server" })

async with aiohttp.ClientSession(cookies=cookies) as session:
    # connect to websocket
    async with session.ws_connect(ws_url) as ws:
        bridge = Bridge("sender", AioTransport(ws)) # type: ignore
        bridge.dst_filter = { "server" }
        bridge.receive_timeout = 1
        asyncio.create_task(sender_task())
        await bridge.run()

print("DONE")

WSServerHandshakeError: 403, message='Invalid response status', url='http://localhost:5000/ws'